In [25]:
import sys
import os
# Add the project root to sys.path
sys.path.append(os.path.abspath('..'))

from Functions.gradientdescent_lib import *
from Functions.crossvalidation_lib import *

In [26]:
np.random.seed(1)
x_train, x_test, y_train, y_test = generateData(100, noise = 0.1)
x_train = x_train.flatten(); x_test = x_test.flatten()


learningRate = 0.05
decay = 0.99
n_features = 4
noIntercept = False
n_folds = 5

optimizer = Optimizers.RMSprop(learningRate, decay, n_features+int(not noIntercept))

gradient_OLS = Gradients.OLS()
gradient_Ridge = Gradients.Ridge(0.01)
gradient_Lasso = Gradients.Lasso(0.01)

GRADIENTS = [gradient_OLS, gradient_Ridge, gradient_Lasso]

for gradient in GRADIENTS:
    model = GradientDescent(n_features, noIntercept=False)
    model.setOptimizer(optimizer)
    model.setGradient(gradient)
    
    Mean_MSE, MSE_scores = cross_validation(n_folds, x_train, y_train, model.evaluation_function())

    print(f'Mean score of {gradient}         = {Mean_MSE}')
    print(f'Standard deviation of {gradient} = {np.std(MSE_scores)}')



Mean score of OLS           = 0.031614850416135085
Standard deviation of OLS   = 0.012517907597978866
Mean score of Ridge         = 0.05389785712636959
Standard deviation of Ridge = 0.019249857371711972
Mean score of Lasso         = 0.048676259528403786
Standard deviation of Lasso = 0.01978313533010979
